In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader,TensorDataset
from torch.optim import RMSprop
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [2]:
features,target = make_classification(n_classes=2,
                                        n_features=10,
                                        n_samples=1000
                                    )

features_train,features_test,target_train,target_test = train_test_split(features,target,test_size=0.1,random_state=1)

In [3]:
torch.manual_seed(0)
np.random.seed(0)

In [4]:
x_train = torch.from_numpy(features_train).float()
y_train = torch.from_numpy(target_train).float().view(-1,1)
x_test = torch.from_numpy(features_test).float()
y_test = torch.from_numpy(target_test).float().view(-1,1)

In [5]:
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet,self).__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(10,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,1),
            torch.nn.Dropout(0.1),
            torch.nn.Sigmoid(),
        )

    def forward(self,x):
        x = self.sequential(x)
        return x

In [6]:
network = SimpleNeuralNet()

In [7]:
criterion = nn.BCELoss()
optimizer = RMSprop(network.parameters())

In [8]:
train_data = TensorDataset(x_train,y_train)
train_loader = DataLoader(train_data,batch_size=100,shuffle=True)

In [9]:
epochs = 5
for epoch in range(epochs):
    for batch_idx,(data,target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()

In [10]:
torch.save(
    {
        "epoch" : epoch,
        "model_state_dict" : network.state_dict(),
        "optimizer_state_dict" : optimizer.state_dict(),
        "loss" : loss,
    },
    "model.pt"
)

In [11]:
network = SimpleNeuralNet()
state_dict = torch.load("model.pt",map_location = torch.device('cpu'))["model_state_dict"]
network.load_state_dict(state_dict,strict=False)
network.eval()

SimpleNeuralNet(
  (sequential): Sequential(
    (0): Linear(in_features=10, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=1, bias=True)
    (5): Dropout(p=0.1, inplace=False)
    (6): Sigmoid()
  )
)